![LangChain](../assets/langchain.svg)

# LangChain Demo

Podríamos decir que es el marco de trabajo o _framework_ con la comunidad más grande en estos momentos. Ofrece capacidades de integración con LLMs en varios lenguajes de programación, siendo el principal Python y JavaScript, y existiendo versiones para otros lenguajes como C#.

Al ser un proyecto comunitario, a veces adolece de mucho caos entre las versiones, aunque últimamente lo están haciendo mejor desde la salida de `LangGraph` (la parte de pago).

## Ejemplo del taller

Vamos a realizar un ejemplo de cómo diseñar e implementar un chatbot impulsado por un modelo de lenguaje grande (LLM, _Large Language Model_), en este caso de Azure OpenAI. Este chatbot podrá mantener una conversación y recordar interacciones previas con un modelo de chat.

Hay que tomar en cuenta que este chatbot que construiremos solo utilizará el modelo de lenguaje para conversar. Más adelante en el taller trataremos otros conceptos relacionados tales como:

 - Generación Aumentada por Recuperación: Que permite una experiencia de chatbot sobre una fuente de datos externa, lo cual es conocido habitualmente por sus siglas en ingles de RAG (_Retrieval Augmented Generation_).
 - Agentes: Construir un chatbot que pueda tomar acciones, algunas veces de forma independiente, para satisfacer un requerimiento específico en un área de dominio.

## Dependencias y Referencias

Primero instalaremos las siguientes dependencias:

 - `langchain-core` → contiene las abstracciones base que impulsan el resto del ecosistema de LangChain. Estas abstracciones están diseñadas para ser modulares y simples, permitiendo que cualquier proveedor implemente la interfaz requerida y se integre fácilmente con el resto del ecosistema.
 - `langchain_openai` → proporciona las integraciones de LangChain para OpenAI a través de su SDK, tanto de la propia empresa detrás de ChatGPT como la versión de Azure OpenAI (o cualquier otra).
 - `langgraph` → es una biblioteca o librería de LangChain diseñada para construir aplicaciones con múltiples actores y estado utilizando modelos de lenguaje grande (LLMs).
 - `python-dotenv` → es una biblioteca de Python que facilita la gestión de variables de entorno en tus proyectos.

Con estas dependencias instaladas, importamos los siguienbtes recursos:

- `import os` → Esta es una biblioteca estándar de Python que proporciona una forma de interactuar con el sistema operativo.
- `from dotenv import load_dotenv` → Importa la capacidad de leer las variables de entorno desde un archivo `.env`. La función `load_dotenv` lee el archivo `.env` y carga las variables definidas en él como variables de entorno del sistema. Esto es útil para gestionar configuraciones sensibles como claves API y credenciales sin hardcodearlas en el código fuente.
- `from langchain_openai import AzureChatOpenAI` → Importa la clase `AzureChatOpenAI` que permite interactuar con los modelos de chat de OpenAI alojados en Azure. Esta clase facilita la configuración y el uso de estos modelos para generar respuestas de chat.

Finalmente leemos las variables de entorno que nos proporcionan los siguientes valores:

- `AZURE_OPENAI_ENDPOINT` → Es la dirección (o URL) desde la cual podemos acceder al servicio de Azure OpenAI que tengamos desplegado en nuestra instancia de la nube de Microsoft.
- `AZURE_OPENAI_DEPLOYMENT_NAME` → Es el nombre del despliegue del modelo (LLM) que vamos a utilizar.
- `AZURE_OPENAI_MODEL_NAME` → Es el nombre (o tipo) del modelo que vamos a utilizar. Por ejemplo `gpt-4o`.
- `AZURE_OPENAI_API_VERSION` → Es la versión del API de Azure con la cual estarmos interactuando con el servicio. Por ejemplo `2024-02-15-preview`.

In [ ]:
%pip install langchain-core langchain_openai langgraph python-dotenv

import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Initialize the model
model = AzureChatOpenAI(    
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    model_name=os.environ["AZURE_OPENAI_MODEL_NAME"],
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),    
)

Primero, usemos el modelo directamente. Los `ChatModels` son instancias de los 'Runnables' de LangChain, lo que significa que exponen una interfaz estándar para interactuar con ellos. Para simplemente llamar al modelo, podemos pasar una lista de mensajes al método `.invoke`, el cual es un método que de forma síncrona se quedará esperando por al respuesta.

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

El modelo por sí solo no tiene ningún concepto de estado. Por ejemplo, si haces una pregunta de seguimiento no sabrá de qué le estás hablando.

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

Para solucionar esto, necesitamos pasar todo el historial de la conversación al modelo. Veamos qué sucede cuando hacemos eso:

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

¡Y ahora podemos ver que obtenemos una buena respuesta!

Esta es la idea básica que sustenta la capacidad de un chatbot para interactuar de manera conversacional. Entonces, ¿cómo implementamos esto de la mejor manera?

## Historial de la Conversación

El componente LangGraph de LangChain implementa una capa de persistencia incorporada, lo que lo hace ideal para aplicaciones de chat que soportan múltiples turnos conversacionales.

Envolver nuestro modelo de chat en una aplicación mínima de LangGraph nos permite persistir automáticamente el historial de mensajes, simplificando el desarrollo de aplicaciones de múltiples turnos.

LangGraph viene con un simple punto de control en memoria, que usamos a continuación. En un entorno real o productivo lo más conveniente es usar diferentes _backends_ de persistencia (por ejemplo, SQLite o Postgres).

Así mismo, a partir de ahora, en vez de usar la función `.invoke` que es síncrona, usaremos la versión `.ainvoke` que es asíncrona y más acorde a un caso de uso real.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}

# Define the (single) node in the graph
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory and create app
app = workflow.compile(checkpointer=MemorySaver())

Ahora necesitamos crear una configuración que pasemos al _runnable_ cada vez que interactuemos con el modelo. Esta configuración contiene información que no es parte de la entrada directamente, pero que sigue siendo útil. En este caso, queremos incluir un `thread_id` para identificar la conversación. Esto se ve así:

In [ ]:
config = {"configurable": {"thread_id": "1"}}

query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

Esto nos permite soportar múltiples hilos de conversación con una sola aplicación, un requisito común cuando tu aplicación tiene múltiples usuarios.

Luego podemos invocar la aplicación:

In [ ]:
config = {"configurable": {"thread_id": "1"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

Podemos ver que si continuamos la conversación preservando el `thread_id` ahora el chatbot tiene memoria de la misma sin necesidad de que pasemos en la llamada todo el historial de la conversación.

In [ ]:
config = {"configurable": {"thread_id": "1"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

**¡Genial!** Nuestro chatbot ahora recuerda cosas sobre nosotros. Si cambiamos la configuración para referenciar un `thread_id` diferente, podemos ver que comienza la conversación desde cero. Es decir, si cambianos el valor del `thread_id` estaríamos realmente creando una conversación nueva, y por tanto el historial de la anterior realmente no existe y el chatbot ya no recuerda nuestro nombre.

Sin embargo, siempre podemos volver a la conversación original (ya que la estamos guardando en una base de datos)

In [ ]:
config = {"configurable": {"thread_id": "2"}}

query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

## Plantillas de _prompts_

Un _prompt_ En este contexto, un prompt es un conjunto de instrucciones o información que se proporciona a un modelo de lenguaje (LLM) para guiar su respuesta. Los _prompts_ pueden incluir mensajes del usuario, mensajes del sistema con instrucciones específicas, o cualquier otra información relevante que ayude al modelo a generar una respuesta adecuada. Por ejemplo, en un chatbot, un _prompt_ podría incluir el historial de la conversación y un mensaje del sistema que le indique al modelo cómo debe comportarse o qué tono debe usar. Esto ayuda a que el modelo entienda mejor el contexto y genere respuestas más coherentes y útiles.

El escribir _prompts_ es tanto un arte como una ciencia, muy basada en el ensayo y el error, y totalmente dependiente de la naturaleza, tipo y versión del modelo de Inteligencia Artificial que estemos utilizando. Tanto es así que existe una disciplina al respecto llamada **_Prompt Engineering_**.

Así, surgen las plantillas de _prompts_, unas construcciones que ayudan a convertir la información cruda del usuario en un formato con el que el modelo de lenguaje (LLM) pueda trabajar. En este caso, la entrada cruda del usuario es solo un mensaje, que estamos pasando al LLM. Ahora hagamos esto un poco más complicado. Primero, agreguemos un mensaje del sistema con algunas instrucciones personalizadas (pero aún tomando mensajes como entrada). Luego, agregaremos más entradas además de los mensajes.

Para agregar un mensaje del sistema, crearemos una `ChatPromptTemplate`. Utilizaremos `MessagesPlaceholder` para pasar todos los mensajes.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Ahora podemos actualizar nuestra aplicación para incorporar esta plantilla:

In [ ]:
async def call_model_with_prompt(state: MessagesState):
    chain = prompt | model
    response = chain.invoke(state)
    return {"messages": response}

workflow_pirate = StateGraph(state_schema=MessagesState)
workflow_pirate.add_edge(START, "model")
workflow_pirate.add_node("model", call_model_with_prompt)
app_pirate = workflow_pirate.compile(checkpointer=MemorySaver())

Creamos un nuevo hilo de conversación para este ejemplo e invocamos la aplicación de la misma manera:

In [ ]:
config = {"configurable": {"thread_id": "3"}}

query = "Hi! I'm Jim."

input_messages = [HumanMessage(query)]
output = await app_pirate.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

Vemos que mantenemos la memoria de la conversación:

In [ ]:
config = {"configurable": {"thread_id": "3"}}

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = await app_pirate.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

**¡Genial!** Ahora hagamos nuestro _prompt_ un poco más complicado. Supongamos que la plantilla del _prompt_ ahora se ve algo así:

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Hemos añadido una nueva entrada de idioma (`language`) al _prompt_. Nuestra aplicación ahora tiene dos parámetros: los mensajes de entrada y el idioma. Debemos actualizar el estado de nuestra aplicación para reflejar esto.

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

async def call_model_with_parameter(state: State):
    chain = prompt | model
    response = await chain.ainvoke(state)
    return {"messages": [response]}

workflow_with_parameter = StateGraph(state_schema=State)
workflow_with_parameter.add_edge(START, "model")
workflow_with_parameter.add_node("model", call_model_with_parameter)
app_with_parameter = workflow_with_parameter.compile(checkpointer=MemorySaver())

Vamos a probarlo haciendo que nuestro chatbot reciba un mensaje en Inglés, pero nos conteste en Español:

In [ ]:
config = {"configurable": {"thread_id": "4"}}

query = "Hi! I'm Bob."

language = "Spanish"

input_messages = [HumanMessage(query)]
output = await app_with_parameter.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

Podemos ver que la memoria preserva el estado del idioma que pasamos, con lo cual no hay que continuar diciéndole que nos hable en Español:

In [ ]:
config = {"configurable": {"thread_id": "4"}}

query = "What is my name?"

input_messages = [HumanMessage(query)]
output = await app_with_parameter.ainvoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

## Gestión del Historial de Conversaciones

Un concepto importante a entender al construir chatbots es cómo gestionar el historial de conversaciones. Si no se gestiona, la lista de mensajes crecerá sin límites y potencialmente desbordará la ventana de contexto del modelo de lenguaje (LLM). Recordemos que los LLMs tienen límites en el número de letras (tokens) que podemos suministrar en las llamadas que realizamos, y por tanto, dentro del propio _prompt_. Así, es importante añadir un paso que limite el tamaño de los mensajes que estás pasando.

Es importante hacer esto **antes** de la plantilla del _prompt_ pero **después** de cargar los mensajes anteriores del historial de mensajes.

Podemos hacer esto añadiendo un paso simple delante del _prompt_ que modifique la clave de los mensajes adecuadamente, y luego envolver esa nueva cadena (`string`) en la clase de historial de mensajes.

LangChain viene con algunos ayudantes incorporados para gestionar una lista de mensajes. En este caso, usaremos el ayudante `trim_messages` para reducir la cantidad de mensajes que estamos enviando al modelo. El `trimmer` nos permite especificar cuántos tokens queremos conservar, junto con otros parámetros como si queremos mantener siempre el mensaje del sistema y si permitimos mensajes parciales:

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages, ToolMessage

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
    end_on=("human", "tool"),
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

Para usarlo en nuestra cadena (`string`), solo necesitamos ejecutar el `trimmer` antes de pasar la entrada de mensajes a nuestro _prompt_:

In [ ]:
async def call_model_with_prompt_and_history(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = await chain.ainvoke(
        {
            "messages": trimmed_messages, 
            "language": state["language"]
        }
    )
    return {"messages": [response]}

workflow_with_prompt_and_history = StateGraph(state_schema=State)
workflow_with_prompt_and_history.add_edge(START, "model")
workflow_with_prompt_and_history.add_node("model", call_model_with_prompt_and_history)
app_with_prompt_and_history = workflow_with_prompt_and_history.compile(checkpointer=MemorySaver())

Ahora, si intentamos preguntarle al modelo nuestro nombre, no lo sabrá ya que recortamos esa parte del historial de chat:

In [ ]:
config = {"configurable": {"thread_id": "5"}}

query = "What is my name?"

language = "English"

input_messages = messages + [HumanMessage(query)]
output = await app_with_prompt_and_history.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

Pero si preguntamos sobre información que está dentro de los últimos mensajes, lo recuerda:

In [ ]:
config = {"configurable": {"thread_id": "6"}}

query = "What math problem did I ask?"

language = "English"

input_messages = messages + [HumanMessage(query)]
output = await app_with_prompt_and_history.ainvoke(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
)
output["messages"][-1].pretty_print()

## _Streaming_

Ahora tenemos un chatbot funcional. Sin embargo, una consideración muy importante de la experiencia de usuario (UX, _User Experience_) para las aplicaciones de chatbot es el _streaming_. Los modelos de lenguaje (LLMs) a veces pueden tardar un tiempo en responder y, para mejorar la experiencia del usuario, una cosa que la mayoría de las aplicaciones hacen es transmitir cada token a medida que se genera. Esto permite al usuario ver el progreso tal como si la Inteligencia Artificial le estuviera escribiendo.

**¡En realidad es muy fácil hacer esto!**

Por defecto, `.stream` en nuestra aplicación LangGraph transmite los pasos de la aplicación; en este caso, el único paso de la respuesta del modelo. El configurar `stream_mode="messages"` nos permite transmitir tokens de salida en su lugar:

In [ ]:
def call_model_with_prompt_and_history_stream(state: State):
    chain = prompt | model
    trimmed_messages = trimmer.invoke(state["messages"])
    response = chain.invoke(
        {
            "messages": trimmed_messages, 
            "language": state["language"]
        }
    )
    return {"messages": [response]}

workflow_with_prompt_and_history_stream = StateGraph(state_schema=State)
workflow_with_prompt_and_history_stream.add_edge(START, "model")
workflow_with_prompt_and_history_stream.add_node("model", call_model_with_prompt_and_history_stream)
app_with_prompt_and_history_stream = workflow_with_prompt_and_history_stream.compile(checkpointer=MemorySaver())


config = {"configurable": {"thread_id": "7"}}

query = "Hi I'm Todd, please tell me a joke."

language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app_with_prompt_and_history_stream.stream(
    {
        "messages": input_messages, 
        "language": language
    },
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")